In [9]:
import librosa
print(librosa.__version__)


0.11.0


In [2]:
import torch
import torch.nn as nn
import torchaudio
import sounddevice as sd
import numpy as np
import librosa
import os

# ========== Define the CNN Model ==========
class EmotionCNN(nn.Module):
    def __init__(self):
        super(EmotionCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(32 * 32 * 32, 128),   # CHANGED from 64 → 128
            nn.ReLU(),
            nn.Linear(128, 8)               # CHANGED from 64 → 128 input
        )

    def forward(self, x):
        return self.model(x)

# ========== Load Model ==========
MODEL_PATH = r"C:\Users\Vihas\Documents\Pytesting\voice_tone\emotion_model.pth"
model = EmotionCNN()
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

# ========== Emotion Label Mapping ==========
emotion_labels = ['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised']

# ========== Mel Spectrogram Extraction ==========
def extract_mel(y, sr=22050):
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128)
    mel_db = librosa.power_to_db(mel, ref=np.max)
    mel_db = mel_db[:, :128]  # Ensure consistent shape
    return mel_db


# ========== Main Loop ==========
print("Press 'r' to record 3 seconds of your voice...")

while True:
    key = input("Press 'r' to record or 'q' to quit: ").strip().lower()

    if key == 'q':
        print("Exiting...")
        break

    elif key == 'r':
        print("Recording for 3 seconds...")
        duration = 3  # seconds
        sr = 22050

        recording = sd.rec(int(duration * sr), samplerate=sr, channels=1)
        sd.wait()
        y = recording.flatten()

        mel = extract_mel(y, sr)

        # Normalize and prepare for model
        mel_tensor = torch.tensor(mel).unsqueeze(0).unsqueeze(0).float()  # Shape: [1, 1, 128, 128]

        # Predict
        with torch.no_grad():
            output = model(mel_tensor)
            predicted = torch.argmax(output, dim=1).item()
            emotion = emotion_labels[predicted]

        print(f"Predicted Emotion: {emotion}")


C:\Users\Vihas\AppData\Local\Temp\ipykernel_35524\2155340014.py:32: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(MODEL_PATH))


Press 'r' to record 3 seconds of your voice...
Recording for 3 seconds...
Predicted Emotion: fearful
Recording for 3 seconds...
Predicted Emotion: happy
Recording for 3 seconds...
Predicted Emotion: calm
Recording for 3 seconds...
Predicted Emotion: disgust
Exiting...
